## 第三周：模型训练
__author__ = 'hehuihui@caicloud.io'
#### 前半部分为特征工程；后半部分为模型训练、超参数搜索


In [1]:
from __future__ import division

import numpy as np
import pandas as pd
import talib

import keras
import tflearn

# 设置随机数种子，使得训练结果可重现
np.random.seed(0)

# 设置屏幕最大可显示的pandas行数与列数，方便查看数据
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

Using TensorFlow backend.


#### 读取数据

In [2]:
df = pd.read_csv("../data/000002.csv")
df.drop('code', axis=1, inplace=True)

#### a)	前5、10、20、40个交易日内的平均成交量、上涨时的平均成交量、下跌时的平均成交量


In [3]:
# 计算前5、10、20、40个交易日内的平均成交量
for w in [5, 10, 20, 40]:
    column = 'volume-mean-%d' % (w)
    df[column] = df['volume'].rolling(window=w).mean()

# 计算当天的涨跌幅
pct_change = df['close'].pct_change().fillna(0)

# 计算当天是否为涨，若为涨则取成交量，否则设为nan
condition_up = pct_change.apply(lambda x: 1 if x > 0 else 0)
volume_up = df['volume'] * condition_up

# 计算当天是否为跌，若为跌则取成交量，否则设为nan
condition_down = pct_change.apply(lambda x: 1 if x <= 0 else 0)
volume_down = df['volume'] * condition_down

In [4]:
def get_volume_mean(volume, condition, window):
    ''' 统计前N日上涨或下跌时的平均成交量
    Args:
    -------------------------------------------
        volume: pd.Series, 上涨(或下跌)时的成交量，若当天不是上涨(或下跌)，则成交量为np.nan
        condition: pd.Series, 当天是否为上涨(或下跌)
        window: int, 时间窗口 N=(5, 10, 20, 40)
    Returns:
    -------------------------------------------
        volume_mean: pd.Series, 前N日上涨(或下跌)时的平均成交量
    '''
    # 平均成交量，最开始的N天设为nan
    volume_mean = [np.nan] * window
    # 遍历每一天，计算前N天的平均成交量
    for i in range(window, len(volume)):
        # 前N天内上涨或下跌的天数（注意：num可能为0）
        num = sum(condition[i-window:i])
        # 前N天内上涨或下跌的平均成交量
        value = np.sum(volume[i-window:i]) / max(1, num)
        volume_mean.append(value) 
    return volume_mean


# 计算前5、10、20、40个交易日内，上涨或下跌时的平均成交量
for w in [5, 10, 20, 40]:
    # 上涨时的平均成交量
    column = 'volume-up-mean-%d' % (w)
    df[column] = get_volume_mean(volume_up, condition_up, w)
    
    # 下跌时的平均成交量
    column = 'volume-down-mean-%d' % (w)
    df[column] = get_volume_mean(volume_down, condition_down, w)

# print df

#### b)	前5、10、20、40个交易日区间内的(收盘价-开盘价)/开盘价、(最高价-开盘价) /开盘价、(最低价-开盘价) /开盘价、(最高价-收盘价) /开盘价、(最低价-收盘价) /开盘价


In [5]:
for w in [5, 10, 20, 40]:
    df['close-open-%d' % w] = (df['close'] - df['open']) / df['open']
    df['high-open-%d' % w] = (df['high'] - df['open']) / df['open']
    df['low-open-%d' % w] = (df['low'] - df['open']) / df['open']
    df['high-close-%d' % w] = (df['high'] - df['close']) / df['open']
    df['low-close-%d' % w] = (df['low'] - df['close']) / df['open']

# 此指标衡量的是资金流入与流出之差（近似）
for w in [1, 3, 5, 10]:
    acc_amount = (df['high'] + df['low'] - df['open'] - df['close']) * df['volume']
    df['acc-amount-%d' % w] = acc_amount.rolling(window=w).sum()

#### d) 使用talib生成MACD、KDJ、RSI、EMV指标


In [6]:
# MACD
df['MACD'], df['MACD-signal'], df['MACD-hist'] = \
    talib.MACD(df['close'].values, fastperiod=12, slowperiod=26, signalperiod=9)

# KDJ
K, D = talib.STOCH(df['high'].values, df['low'].values, df['close'].values, 
                       fastk_period=9, slowk_period=3, slowd_period=3)
J = 3*K - 2*D

# KDJ
df['KDJ-K'] = K
df['KDJ-D'] = D
df['KDJ-J'] = J

# RSI
for period in [6, 12, 24]:
    df['RSI-%d' % (period)] = talib.RSI(df['close'].values, timeperiod=period)

# EMV

# 计算公式(https://www.joinquant.com/post/150)
# 1.A=（今日最高+今日最低）/2
#   B=（前日最高+前日最低）/2
#   C=今日最高-今日最低
# 2.EM=（A-B）*C/今日成交额
# 3.EMV=N日内EM的累和

diff = (df['high'] + df['low']) - (df['high'].shift(1) + df['low'].shift(1))
mv = 1e8 * diff * (df['high'] - df['low']) / df['volume']
df['EMV'] = mv.rolling(window=14).mean()
df['MAEMV'] = df['EMV'].rolling(window=9).mean()

## 特征选择

在机器学习的实际应用中，特征数量往往较多，其中可能存在不相关的特征，特征之间也可能存在相互依赖，容易导致：<br/>
- 特征个数越多，分析特征、训练模型所需的时间也就越长
- 特征个数越多，容易引起“维度灾难”，模型也会越复杂，其推广能力会下降。

特征选择能剔除不相关（irrelevant）或亢余（redundant）的特征，从而达到减少特征个数，提高模型精确度，减少运行时间的目的。<br/>
另一方面，选取出真正相关的特征简化了模型，是研究人员易于理解数据产生的过程

In [7]:
# 生成标签
# 注意此处的移位是-1，label是未来一天的收盘价➗当天的收盘价
df['label'] = (df['close'].shift(-1) / df['close']).apply(lambda x: 1 if x > 1 else 0)

# 过滤缺失值
new_df = df.dropna(how='any').reset_index(drop=True)

# 获取特征与标签
dataset = new_df.iloc[:, 1:].values

In [8]:
def split_train_test(dataset, split_ratio=0.8):
    ''' 划分训练集和测试集
    # Args
        dataset: np.ndarray, 待划分的数据集
        split_ratio: float, 训练集所占比例
    # Returns
        X_train, Y_train: np.ndarray, 训练集中的特征和标签
        X_test, Y_test: np.ndarray, 测试集中的特征和标签
    '''
    N = int(len(dataset) * split_ratio) # 训练集与测试集的分界点
    
    X_train = dataset[:N, :-1]
    y_train = dataset[:N, -1]
    
    X_test = dataset[N:, :-1]
    y_test = dataset[N:, -1]
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = split_train_test(dataset, split_ratio=0.8)

In [9]:
# 特征归一化
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_transformed = scaler.fit_transform(X_train)
X_test_transformed = scaler.transform(X_test)

#### 卡方统计量

In [10]:
# 对于分类问题，可使用的特征选择方法: chi2, f_classif, mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

K = 6    # 选择最好的K个特征
selector = SelectKBest(chi2, k=K)
X_train_selected = selector.fit_transform(X_train_transformed, y_train)
X_test_selected = selector.transform(X_test_transformed)

/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/scipy/sparse/compressed.py:130: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.indices) != 1 or np.rank(self.indptr) != 1:
/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/scipy/sparse/coo.py:200: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.row) != 1 or np.rank(self.col) != 1:


### 生成LSTM所需的训练集与测试集

In [11]:
def create_LSTM_dataset(X, y, time_steps=20):
    ''' 生成LSTM训练所需的数据集格式
    # Args
        X: np.2D-array, 特征矩阵
        y: np.1D-array, 标签矩阵
        time_steps: int, 使用多少天的数据作为特征
    # Returns:
        X_dataset, y_dataset: (np.3D-array, np.2D-array)，特征数据集、标签数据集
    '''
    X_list, y_list = [], []
    assert(len(X) == len(y))
    for i in range(len(X) - time_steps - 1):
        X_list.append(X[i:(i + time_steps), :])
        y_list.append(y[(i + 1):(i + time_steps + 1)])
    return np.array(X_list), np.array(y_list)

train_X, train_y = create_LSTM_dataset(X_train_selected, y_train)
test_X, test_y = create_LSTM_dataset(X_test_selected, y_test)

#print train_X.shape, train_y.shape, test_X.shape, test_y.shape

In [12]:
train_X.shape, train_y.shape

((799, 20, 6), (799, 20))

### 定义LSTM模型

In [13]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM, Dropout, BatchNormalization

def create_LSTM_model(lstm_neurons, dense_neurons, input_shape=[20, K],
                     initializers='glorot_uniform', dropout_prob=0.5, 
                     optimizers='adam', activations='sigmoid'):
    ''' 定义LSTM模型结构
    Args:
    ----------------------------------------------------------------------
        lstm_neurons: int, LSTM层的神经元数量
        dense_neurons: int, Dense全连接层的神经元数量
        input_shape: 2D-array, 输入层的数据维度
        initializers: str, 权重初始化方法
        dropout_prob: float, Dropout层的保留比例
        optimizers: str, 优化方法
        activations: str, 激活函数
    
    Returns:
    ----------------------------------------------------------------------
        model: keras.models.Model, 定义好的深度学习模型
    '''
    inputs = Input(shape=input_shape)

    x = LSTM(lstm_neurons, kernel_initializer=initializers, dropout=0.5)(inputs)
    #x = BatchNormalization()(x)
    x = Dense(dense_neurons, kernel_initializer=initializers, activation='relu')(x)

    if 0 < dropout_prob < 1:
        x = Dropout(dropout_prob)(x)

    outputs = Dense(input_shape[0], kernel_initializer=initializers, activation=activations)(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers, loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [14]:
# 单独训练一次试试
model = create_LSTM_model(input_shape=[20, K], lstm_neurons=128, dense_neurons=64)
history = model.fit(train_X, train_y, 
                    epochs=20, verbose=2, batch_size=16, shuffle=False, 
                    validation_data=(test_X, test_y))

Train on 799 samples, validate on 185 samples
Epoch 1/20
6s - loss: 0.6942 - acc: 0.4982 - val_loss: 0.6933 - val_acc: 0.5008
Epoch 2/20
5s - loss: 0.6932 - acc: 0.5054 - val_loss: 0.6927 - val_acc: 0.5111
Epoch 3/20
4s - loss: 0.6925 - acc: 0.5191 - val_loss: 0.6923 - val_acc: 0.5222
Epoch 4/20
4s - loss: 0.6914 - acc: 0.5218 - val_loss: 0.6914 - val_acc: 0.5308
Epoch 5/20
3s - loss: 0.6904 - acc: 0.5263 - val_loss: 0.6907 - val_acc: 0.5351
Epoch 6/20
3s - loss: 0.6883 - acc: 0.5422 - val_loss: 0.6902 - val_acc: 0.5389
Epoch 7/20
3s - loss: 0.6851 - acc: 0.5495 - val_loss: 0.6887 - val_acc: 0.5446
Epoch 8/20
3s - loss: 0.6817 - acc: 0.5571 - val_loss: 0.6882 - val_acc: 0.5451
Epoch 9/20
4s - loss: 0.6799 - acc: 0.5572 - val_loss: 0.6881 - val_acc: 0.5459
Epoch 10/20
3s - loss: 0.6788 - acc: 0.5612 - val_loss: 0.6880 - val_acc: 0.5468
Epoch 11/20
3s - loss: 0.6759 - acc: 0.5728 - val_loss: 0.6856 - val_acc: 0.5514
Epoch 12/20
3s - loss: 0.6770 - acc: 0.5668 - val_loss: 0.6832 - val_acc

#### 自定义参数搜索器

In [ ]:
# 待搜索的参数
param_grid = dict(lstm_neurons=[64, 128, 256],
                  dense_neurons=[64, 128, 256],
                  dropout_prob=[0.25, 0.5, 0.75],
                  optimizers=['adam', 'adagrad', 'rmsprop'],
                  activations=['sigmoid', 'relu', 'tanh'])

# 生成各参数的所有组合
from itertools import product
def grid_search(param_grid):
    keys, values = zip(*param_grid.items())
    for v in product(*values):
        params = dict(zip(keys, v))
        yield params

result = list()
for params in grid_search(param_grid):
    # 使用不同的参数来定义LSTM模型
    model = create_LSTM_model(**params)
    # 枚举batch_size
    for batch_size in [8, 16, 32]:
        # 记录训练过程
        history = model.fit(train_X, train_y, 
                    epochs=10, verbose=1, batch_size=batch_size, shuffle=False, 
                    validation_data=(test_X, test_y)).history
        
        # 记录训练结果，比如acc, loss, val_acc, val_loss
        for key in history.keys():
            if key.find('loss') >= 0:
                params['best_' + key] = min(history[key][-3:])
            else:
                params['best_' + key] = max(history[key][-3:])
        
        params['batch_size'] = batch_size
        result.append(params)
        print params

Train on 799 samples, validate on 185 samples
Epoch 1/10
799/799 [==============================] - 9s - loss: 0.6939 - acc: 0.5027 - val_loss: 0.6932 - val_acc: 0.5122

#### 使用GridSearchCV来进行参数搜索

In [ ]:
# 此部分在Mac上运行会报错(错误信息：使用_umath_linalg.so插件时python意外退出)。待排查原因
# from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
# from keras.wrappers.scikit_learn import KerasClassifier

# tscv = TimeSeriesSplit(n_splits=2)

# model = KerasClassifier(build_fn=create_LSTM_model, epochs=10, batch_size=16, verbose=2)
# param_grid = dict(lstm_neurons=[64, 128],
#                   dense_neurons=[64, 128, 256])

# grid_cv = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, n_jobs=4)
# grid_cv.fit(train_X, train_y)